In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://uscbathletics.com/sports/mens-basketball/stats'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [4]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [5]:
#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [6]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data

['04', 'Gaines, Kenney', '24', '0', '571', '23.8', '105', '218', '.482', '7', '28', '.250', '83', '115', '.722', '300', '12.5', '30', '74', '104', '4.3', '37', '66', '61', '44', '6', 'View Bio']
['14', 'Eason, Dominic', '23', '23', '518', '22.5', '93', '166', '.560', '37', '87', '.425', '41', '64', '.641', '264', '11.5', '26', '56', '82', '3.6', '27', '17', '28', '15', '20', 'View Bio']
['32', 'Price, Kevaughn', '24', '24', '507', '21.1', '91', '187', '.487', '18', '42', '.429', '46', '75', '.613', '246', '10.3', '41', '23', '64', '2.7', '65', '30', '29', '21', '5', 'View Bio']
['05', 'Norton, Hudson', '24', '1', '398', '16.6', '71', '147', '.483', '21', '56', '.375', '54', '79', '.684', '217', '9.0', '43', '59', '102', '4.3', '47', '26', '18', '14', '5', 'View Bio']
['07', 'Ralat, Alejandro', '23', '23', '477', '20.7', '67', '168', '.399', '21', '62', '.339', '46', '60', '.767', '201', '8.7', '15', '17', '32', '1.4', '47', '67', '53', '34', '1', 'View Bio']
['11', 'Sfanos, Tasso', '24

In [7]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/ucsb_player_stats.csv', index=False, encoding = "utf-8")